$ Pregunta \ 1 $

$a)$ Primero que todo definiremos que es el sesgo inductivo y que es sesgo en la predicción.

$Sesgo \ inductivo:$ Es el conjunto de suposiciones que se utilizan para predecir soluciones a un problema especifico. Por ejemplo, un sesgo inductivo podría ser asumir que la solución debe ser una regresion lineal o polinomica.

$Sesgo \ en \ la \ predicción:$ Corresponde al error que se comete en el output de un algoritmo. Viene dado por la formula:

$ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ [E[\hat{f}(x)] - f(x)]^2 $

$ \ $ 

De esta manera, podriamos decir que ambas definiciones son muy distintas, sin embargo, es posible establecer una relación entre ambas, pues es mi sesgo inductivo el que define mi $f$ a utilizar y como este $f$ influye en el sesgo de predicción, entonces podemos decir que el sesgo en la predicción se ve fuertemente influido por mi sesgo inductivo.  

Para verlo más en simple pondremos un ejemplo. Supongamos que tengo datos que tienen una forma muy parecida a una recta y pongamosnos los siguientes 2 casos:

i) Mi sesgo inductivo es el de un regresión lineal. Mi error en la predicción sería pequeño. 

ii) Mi sesgo inductivo es el de una regresión formada por una suma de senos y cosenos. Mi error en la predicción sería grande. 

Así de esta manera podemos ejemplificar claramente la relación que existe entre ambos sesgos. 

$  $ 

$  $ 

$  $ 

$b)$ Los sesgos inductivos para k-NN y para los arboles de decisión son los siguientes:

$ k-NN:$ El sesgo inductivo de este algoritmo es que los datos estan dividos en grupos de caracteristicas similares, es por esto que en este algoritmo lo que se hace para predecir un dato, es que dadas sus caracteristicas (o features) se observan los k vecinos más cercanos y entre estos vecinos se ve el grupo que más se repite y ese grupo es el que se le asigna al dato. 

$ Arboles \ basados \ en \ ganancia \ de \ informacion: $ El sesgo inductivo de este algoritmo es generar arboles que sean más anchos que profundos, pues este algoritmo genera este tipo de arboles. 

La relación entre el sesgo de k-NN y la clasificación realizada en un nodo hoja de un arbol es que en ambos casos los que se hace es analizar sobre un grupo de opciones a cual es el que se parece más y partir de eso, clasificarlo como ese "grupo más similar". 


$ \ $ 

$ \ $

$ \ $

$ c) $ La base de datos que vamos a utilizar en esta pregunta es "2014_Financial_Data.csv" es decir, la base de datos que
nos dieron por enunciado (archivo que adjuntamos junto a esta pregunta).

Se implementaron dos arboles de decisión. Uno basado en la mínima ganancia de información y otro basado en la máxima ganancia de información, en donde más adelante al obtener resultados se realiza su analisis respectivo. 

Pero antes de implementar estos arboles debemos decir que como estamos trabajando con variables númericas a cada columna de información se le tiene que realizar el siguiente preprocesamiento. 

Se debe elegir un $n$ tal que al separar una columna en los datos que son menor o igual $n$ y en los que son mayor que $n$ nos genere la máxima ganancia de información posible. Para poder determinar este $n$ lo que hicimos fue que en cada columna consideramos como candidatos a los mismos valores que conformaban a la columna. Una vez realizado esto, procedemos a aplicar el algoritmo de minima/maxima información.  Esto se debe realizar para cada split.

$ \ $

$ \ $

$ \ $

In [1]:
import math
import time
import numpy as np
import pandas as pd

In [2]:
def son_todos_iguales(clase):
    if len(clase) == 1:
        return True
    primer_elemento = clase[0]
    for i in range(len(clase)-1):
        if clase[i+1] != primer_elemento:
            return False
    return True

In [3]:
def calcular_separador(columna, clase):
    
    lista_entropia = []
    lista_valores = []
    
    cantidad_positivos_padre = 0
    cantidad_negativos_padre = 0

    for i in range(len(columna)):
        if clase[i] == 1:
            cantidad_positivos_padre = cantidad_positivos_padre + 1
        else:
            cantidad_negativos_padre = cantidad_negativos_padre + 1

    p_positivos_padre = cantidad_positivos_padre/(cantidad_positivos_padre+cantidad_negativos_padre)
    p_negativos_padre = cantidad_negativos_padre/(cantidad_positivos_padre+cantidad_negativos_padre)

    entropia_padre = 0

    if p_positivos_padre != 0 and p_negativos_padre != 0:
        entropia_padre = entropia_padre - p_positivos_padre*math.log2(p_positivos_padre)
        entropia_padre = entropia_padre - p_negativos_padre*math.log2(p_negativos_padre)
    
    # CALCULO DE ENTROPIA PARA HIJOS
    
    for i in range(len(columna)):
        
        lista_menor = []
        lista_mayor = []
        
        for j in range(len(columna)):
            if columna[j] <= columna[i]:
                lista_menor.append([columna[j], j])
            else:
                lista_mayor.append([columna[j], j])
                
        
        p_hijo_mayor = len(lista_mayor)/(len(lista_mayor)+len(lista_menor))
        p_hijo_menor = len(lista_menor)/(len(lista_mayor)+len(lista_menor))
        
        # Entropia hijo menor
        
        cantidad_positivos_hijo_menor = 0
        cantidad_negativos_hijo_menor = 0
        
        for h1 in lista_menor:
            if clase[h1[1]] == 1:
                cantidad_positivos_hijo_menor = cantidad_positivos_hijo_menor + 1
            else:
                cantidad_negativos_hijo_menor = cantidad_negativos_hijo_menor + 1
        
        if cantidad_positivos_hijo_menor != 0 and cantidad_negativos_hijo_menor != 0:
            p_positivos_hijo_menor = cantidad_positivos_hijo_menor/(cantidad_positivos_hijo_menor + cantidad_negativos_hijo_menor)
            p_negativos_hijo_menor = cantidad_negativos_hijo_menor/(cantidad_positivos_hijo_menor + cantidad_negativos_hijo_menor)
        else:
            p_positivos_hijo_menor = 0
            p_negativos_hijo_menor = 0
            
        entropia_hijo_menor = 0 
        
        if p_positivos_hijo_menor != 0 and p_negativos_hijo_menor != 0:
            
            entropia_hijo_menor = entropia_hijo_menor - p_positivos_hijo_menor*math.log2(p_positivos_hijo_menor)
            entropia_hijo_menor = entropia_hijo_menor - p_negativos_hijo_menor*math.log2(p_negativos_hijo_menor)
        
        # Entropia_hijo_mayor 
        
        cantidad_positivos_hijo_mayor = 0
        cantidad_negativos_hijo_mayor = 0
        
        for h2 in lista_mayor:
            if clase[h2[1]] == 1:
                cantidad_positivos_hijo_mayor = cantidad_positivos_hijo_mayor + 1
            else:
                cantidad_negativos_hijo_mayor = cantidad_negativos_hijo_mayor + 1
        
        if cantidad_positivos_hijo_mayor != 0 and cantidad_negativos_hijo_mayor != 0:
            p_positivos_hijo_mayor = cantidad_positivos_hijo_mayor/(cantidad_positivos_hijo_mayor + cantidad_negativos_hijo_mayor)
            p_negativos_hijo_mayor = cantidad_negativos_hijo_mayor/(cantidad_positivos_hijo_mayor + cantidad_negativos_hijo_mayor)
        else:
            p_positivos_hijo_mayor = 0
            p_negativos_hijo_mayor = 0
            
        entropia_hijo_mayor = 0
        
        if p_positivos_hijo_mayor != 0 and p_negativos_hijo_mayor != 0:
            entropia_hijo_mayor = entropia_hijo_mayor  - p_positivos_hijo_mayor*math.log2(p_positivos_hijo_mayor)
            entropia_hijo_mayor = entropia_hijo_mayor  - p_negativos_hijo_mayor*math.log2(p_negativos_hijo_mayor)
        
        entropia_total = entropia_padre - p_hijo_mayor*entropia_hijo_mayor - p_hijo_menor*entropia_hijo_menor
        
        
        lista_entropia.append(entropia_total)
        lista_valores.append(columna[i])
        
    return [lista_entropia, lista_valores]

In [4]:
def calcular_arbol_maximo(columnas, clase, profundidad, info):
    
    if son_todos_iguales(clase) == True:
        profundidad_maximo.append(profundidad)
        return [clase[0], profundidad]
    
    lista_entropias = []
    lista_separadores = []
    
    for i in range(len(columnas)):
        output = calcular_separador(columnas[i], clase)
        max_ganancia = max(output[0])
        indice = output[0].index(max_ganancia)
        separador = output[1][indice]
        
        lista_entropias.append(max_ganancia)
        lista_separadores.append(separador)
    
    indice_elegido = lista_entropias.index(max(lista_entropias))
    
    if info[0] == indice_elegido and info[1] == lista_separadores[indice_elegido]:
        if np.mean(clase) >= 0.5:
            return [1, profundidad]
        else:
            return [0, profundidad]
    
    
    nuevas_columnas_1 = []
    nuevas_columnas_2 = []
    
    nueva_clase_1 = []
    nueva_clase_2 = []
    
    indices_1 = []
    indices_2 = []
    
    for j in range(len(columnas[indice_elegido])):
        if columnas[indice_elegido][j] <= lista_separadores[indice_elegido]:
            nueva_clase_1.append(clase[j])
            indices_1.append(j)
        else:
            nueva_clase_2.append(clase[j])
            indices_2.append(j)
    
    for i in range(len(columnas)):
        lista_1 = []
        for k1 in indices_1:
            lista_1.append(columnas[i][k1])
        lista_2 = []
        for k2 in indices_2:
            lista_2.append(columnas[i][k2])
        nuevas_columnas_1.append(lista_1)
        nuevas_columnas_2.append(lista_2)

    info_nueva = [indice_elegido, lista_separadores[indice_elegido]]

    if info[0] == indice_elegido and info[1] == lista_separadores[indice_elegido]:
        if np.mean(clase) >= 0.5:
            return [1, profundidad]
        else:
            return [0, profundidad]
    else:
        if len(nueva_clase_1) == 0:
            if np.mean(nueva_clase_1) > 0.5:
                return [1, profundidad]
            else:
                return [0, profundidad]
        elif len(nueva_clase_2) == 0:
            if np.mean(nueva_clase_2) > 0.5:
                return [1, profundidad]
            else:
                return [0, profundidad]
        else:
            return [calcular_arbol_maximo(nuevas_columnas_1, nueva_clase_1, profundidad + 1, info_nueva),
            calcular_arbol_maximo(nuevas_columnas_2, nueva_clase_2, profundidad + 1, info_nueva),
            info_nueva]

In [5]:
def calcular_arbol_minimo(columnas, clase, profundidad, info):
    
    if son_todos_iguales(clase) == True:
        profundidad_minimo.append(profundidad)
        return [clase[0], profundidad]
    
    lista_entropias = []
    lista_separadores = []
    
    for i in range(len(columnas)):
        output = calcular_separador(columnas[i], clase)
        max_ganancia = max(output[0])
        indice = output[0].index(max_ganancia)
        separador = output[1][indice]
        
        lista_entropias.append(max_ganancia)
        lista_separadores.append(separador)
    
    indice_elegido = lista_entropias.index(min(lista_entropias))
    
    nuevas_columnas_1 = []
    nuevas_columnas_2 = []
    
    nueva_clase_1 = []
    nueva_clase_2 = []
    
    indices_1 = []
    indices_2 = []
    
    for j in range(len(columnas[indice_elegido])):
        if columnas[indice_elegido][j] <= lista_separadores[indice_elegido]:
            nueva_clase_1.append(clase[j])
            indices_1.append(j)
        else:
            nueva_clase_2.append(clase[j])
            indices_2.append(j)
    
    for i in range(len(columnas)):
        lista_1 = []
        for k1 in indices_1:
            lista_1.append(columnas[i][k1])
        lista_2 = []
        for k2 in indices_2:
            lista_2.append(columnas[i][k2])
        nuevas_columnas_1.append(lista_1)
        nuevas_columnas_2.append(lista_2)

    info_nueva = [indice_elegido, lista_separadores[indice_elegido]]
    
    if info[0] == indice_elegido and info[1] == lista_separadores[indice_elegido]:
        if np.mean(clase) >= 0.5:
            return [1, profundidad]
        else:
            return [0, profundidad]
    else:
        if len(nueva_clase_1) == 0:
            if np.mean(nueva_clase_1) > 0.5:
                return [1, profundidad]
            else:
                return [0, profundidad]
        elif len(nueva_clase_2) == 0:
            if np.mean(nueva_clase_2) > 0.5:
                return [1, profundidad]
            else:
                return [0, profundidad]
        else:
            return [calcular_arbol_minimo(nuevas_columnas_1, nueva_clase_1, profundidad + 1, info_nueva),
                    calcular_arbol_minimo(nuevas_columnas_2, nueva_clase_2, profundidad + 1, info_nueva),
                    info_nueva]

In [6]:
def calcular_valor(indice, resultado, columnas):
    if len(resultado) <= 2:
        return resultado[0]
    else:
        seleccion = resultado[2][0]
        
        if columnas[seleccion][indice] <= resultado[2][1]:
            return calcular_valor(indice, resultado[0], columnas)
        else:
            return calcular_valor(indice, resultado[1], columnas)

In [7]:
archivo = pd.read_csv("2014_Financial_Data.csv")

In [8]:
nombres = []
parte_columnas = []
columnas_testeo = []
columnas_entrenamiento = []

for row in archivo:
    nombres.append(row)

# Eliminamos el primer elemento

nombres = nombres[1:len(nombres)]

N = 350
M = 10
valor_inicial = 0
valor_final = valor_inicial + M

nombres = nombres[valor_inicial:valor_final]

# Eliminamos los datos nan

nuevo_archivo = archivo.dropna(subset=nombres)

for k in nombres:
    parte_columnas.append(nuevo_archivo[k])

parte_clases = nuevo_archivo['Class']

for i in range(len(parte_columnas)):
    lista = []
    for j in parte_columnas[i]:
        lista.append(j)
    columnas_entrenamiento.append(lista[0:N])
    columnas_testeo.append(lista[N:len(lista)])

clases_totales = [] 

for i in parte_clases:
    clases_totales.append(i)
    
clase_entrenamiento = clases_totales[0:N]
clase_testeo = clases_totales[N:len(clases_totales)]

In [9]:
profundidad_maximo = []
profundidad_minimo = []

In [10]:
output_maximo = calcular_arbol_maximo(columnas_entrenamiento, clase_entrenamiento, 0, ['A', 'A']) 

In [11]:
# Aciertos en testeo

lista_predicciones = []

for j in range(len(clase_testeo)):
    lista_predicciones.append(calcular_valor(j, output_maximo, columnas_testeo))
    
lista_coincidencias = []

for j in range(len(clase_testeo)):
    if lista_predicciones[j] == clase_testeo[j]:
        lista_coincidencias.append(1)
    else:
        lista_coincidencias.append(0)

print("El porcentaje de acierto en el testeo es: ", np.mean(lista_coincidencias)*100)
print("La profundidad maxima del arbol es:", max(profundidad_maximo))
print("La cantidad de hojas del arbol es:", len(profundidad_maximo))

El porcentaje de acierto en el testeo es:  55.0450170754424
La profundidad maxima del arbol es: 15
La cantidad de hojas del arbol es: 69


In [12]:
output_minimo = calcular_arbol_minimo(columnas_entrenamiento, clase_entrenamiento, 0, ['A', 'A']) 

C:\Users\56977\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\56977\Anaconda3\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [13]:
# Aciertos en testeo

lista_predicciones = []

for j in range(len(clase_testeo)):
    lista_predicciones.append(calcular_valor(j, output_minimo, columnas_testeo))
    
lista_coincidencias = []

for j in range(len(clase_testeo)):
    if lista_predicciones[j] == clase_testeo[j]:
        lista_coincidencias.append(1)
    else:
        lista_coincidencias.append(0)

print("El porcentaje de acierto en el testeo es: ", np.mean(lista_coincidencias)*100)
print("La profundidad maxima del arbol es:", max(profundidad_minimo))
print("La cantidad de hojas del arbol es:", len(profundidad_minimo))

El porcentaje de acierto en el testeo es:  51.87829866501087
La profundidad maxima del arbol es: 38
La cantidad de hojas del arbol es: 91


$ $

$ $

$ Analisis \ de \ los \ resultados: $ 

Como se observa para un arbol basado en la máxima ganancia de información se obtiene un arbol de profundidad baja (15) y con muchas hojas (69) en relación a su profundidad (69/15 = 4,6) y en el arbol basado en la minima ganancia de información se obtiene un arbol de profundidad alta (38) y con una cantidad de hojas (91) que resulta en un proporcion hojas/profundidad (91/38 = 2,4) menor que la de un arbol basado en la maxima ganancia de información.

Así se corrobora el hecho de que el sesgo inductivo de los arboles basados en minima ganancia de información es generar arboles profundos y que el sesgo inductivo de los arboles basados en la maxima ganancia de información generan arboles poco profundos y anchos.

Ahora, dado a que el porcentaje de acierto en testeo para el arbol basado en la minima ganancia de información es de $51.88$ % y el porcentaje de acierto en testeo para el arbol basado en la máxima ganancia de información es de $55.05$ % entonces decimos que el nuevo sesgo inductivo NO es mejor que el de un arbol tradicional.